# Reading the datasets

In [1]:
import pandas as pd
import feather

### Accident Dataset
The US road accident dataset is used as the primary dataset for the project which is a countrywide traffic accident dataset obtained from Kaggle (https://www.kaggle.com/sobhanmoosavi/us-accidents) referencing papers from Moosavi (2019). The period of data entries ranges from February 2016 to December 2020. The aim of this project is to determine the features which affect the severity of an accident, and to predict it given a set of features.

In [2]:
FILENAME = "Datasets/US_Accidents_Dec20_Updated"
data = pd.read_csv(open("{}.csv".format(FILENAME),'rU'), encoding='utf-8', engine='python')

C:\Users\liyoa\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 'U' mode is deprecated
  


### Census Dataset
A supplementary dataset records Census data collected by the U.S. Census Bureau (2019). Specifically, the data is gathered from the American Community Survey (ACS) providing census data over a 5 year estimate (2015-2019).

In [3]:
import censusdata

county = censusdata.download('acs5', 2019, censusdata.censusgeo([('county', '*')]),
                                   ['DP05_0001E',  'DP03_0019PE','DP03_0021PE','DP03_0022PE','DP03_0062E'],
                                   tabletype='profile')
# rename columns
county.columns = ['Population_County','Drive_County','Transit_County','Walk_County','MedianHouseholdIncome_County']
county = county.reset_index()
# extract county name and convert them to lowercase
county['County'] = county['index'].apply(lambda x : x.name.split(' County')[0].split(',')[0]).str.lower()
county['State'] = county['index'].apply(lambda x : x.name.split(', ')[1]).str.lower()
county.head(3)

,index,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County,County,State
0,"Fayette County, Illinois: Summary level: 050, ...",21565,85.2,0.8,2.5,46650,fayette,illinois
1,"Logan County, Illinois: Summary level: 050, st...",29003,80.3,0.4,2.9,57308,logan,illinois
2,"Saline County, Illinois: Summary level: 050, s...",23994,84.4,0.9,1.5,44090,saline,illinois


In [4]:
# Convert all county values in data to lowercase

data['State'] = data['State'].str.lower() 
data['County'] = data['County'].str.lower()

counties = []
for i in data['County'].values:
    counties.append(i.replace(' county', ''))
data['County'] = counties

In [5]:
states = pd.read_csv('Datasets/states.csv') # a csv file with all the US states and their abbreviation 
states.index = states['Abbreviation'].map(lambda value: value.lower())
states = states.drop('Abbreviation',axis=1)
states.head(2)

,State
Abbreviation,
al,Alabama
ak,Alaska


In [6]:
data['State'] = [states.loc[i, 'State'] for i in data['State'].values]
data['State'] = data['State'].map(lambda value: value.lower())

### Merge Accident and Census Datasets

In [7]:
# Create join column <state>-<county>
def concat_cols(df, cols):
    df['combined'] = df[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
    return df

data = concat_cols(data, ['State', 'County'])
county = concat_cols(county, ['State', 'County'])

# left join df with census data
data = data.merge(county, left_on = 'combined', right_on='combined',how = "left")
data = data.drop(["index", "County_y", "State_y", "combined"],axis = 1)

In [8]:
data.head(2)

,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,...,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Population_County,Drive_County,Transit_County,Walk_County,MedianHouseholdIncome_County
0,A-1,2,2019-05-21 08:29:55,2019-05-21 09:29:40,34.808868,-82.269157,34.808868,-82.269157,0.0,Accident on Tanner Rd at Pennbrooke Ln.,...,False,Day,Day,Day,Day,507003.0,82.1,0.4,1.4,60351.0
1,A-2,2,2019-10-07 17:43:09,2019-10-07 19:42:50,35.090080,-80.745560,35.090080,-80.745560,0.0,Accident on Houston Branch Rd at Providence Br...,...,False,Day,Day,Day,Day,1074475.0,77.0,2.9,1.9,66641.0


In [9]:
# Save the dataset
data.to_feather("Datasets/merged_data.feather")